In [1]:
%load_ext autoreload
%autoreload 2

## The job search recruitment agency!
1. Profiler
2. Recruiter
3. Matcher
4. Cover Letter Writer

In [2]:
import warnings
import os
from dotenv import load_dotenv, find_dotenv
import yaml
from crewai import Crew, Process, Agent, Task
from crewai.tools import tool

In [3]:
_ = load_dotenv(find_dotenv())
warnings.filterwarnings("ignore")
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

Instantiate LLM Observability

In [4]:
from phoenix.otel import register

tracer_provider = register(
    project_name = "crewai",
    endpoint="http://localhost:6006/v1/traces"
)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: crewai
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [5]:
from openinference.instrumentation.crewai import CrewAIInstrumentor

CrewAIInstrumentor().instrument(tracer_provider=tracer_provider)

### Loading tasks and agents from yaml files

In [6]:
from typing import Dict, Tuple
def load_configs(files: Dict[str, str]) -> Tuple[dict, dict]:
    """Loads yml config files from a dictionary of filepaths

    Args:
        files (Dict[str, str]): Dictionary of filepaths containing yml files
        For example:
        {
            'agents': '../config/agents.yml',
            'tasks': '../config/tasks.yml'
        }
    """
    configs = {}
    for config_type, file_path in files.items():
        with open(file_path, 'r') as file:
            configs[config_type] = yaml.safe_load(file)
    return configs['agents'], configs['tasks']

In [7]:
job_search_agents_config, job_search_tasks_config = load_configs(
    {
        'agents': '../config/job_search_agents.yml',
        'tasks': '../config/job_search_tasks.yml',
    }
)

# 1. The Job Search Crew

## Create Pydantic Models for Structured Output

In [8]:
from pydantic import BaseModel, Field
from typing import Literal, List

class WorkExperience(BaseModel):
    project_title: str = Field(..., description="Name of project")
    summary_of_experience: str = Field(..., description="Short summary of experience")
    skills_exhibited: List[str] = Field(..., description="Skills exhibited during this project")

class CandidateProfile(BaseModel):
    name: str = Field(..., description="Candidate's name")
    highest_qualification: Literal["Bachelor's Degree", "Masters Degree", "PhD"] = Field(..., description="Candidate's highest qualification")
    field_of_study: str = Field(..., description="Candidate's field of study for highest qualification")
    institution_of_study: str = Field(..., description="Candidate's institution of study for highest qualification")
    technical_skills: List[str] = Field(..., description="List of candidate's technical skills. For example 'machine learning'.")
    non_technical_skills: List[str] = Field(..., description="List of candidate's non-technical skills. For example 'strategic planning'.")
    years_of_experience: int = Field(..., description="Candidate's years of experience")
    current_job_title: str = Field(..., description="Candidate's current job title")
    work_experiences: List[WorkExperience] = Field(..., description="Candidate's job experience")

class JobListing(BaseModel):
    company_name: str = Field(..., description="Company name")
    job_title: str = Field(..., description="Job title")
    job_description: str = Field(..., description="Job description")
    required_technical_skills: List[str] = Field(..., description="List of required technical skills. For example 'machine learning'.")
    required_non_technical_skills: List[str] = Field(..., description="List of required non-technical skills. For example 'strategic planning'.")
    years_of_experience_required: int = Field(..., description="Years of experience required")
    reason_for_candidate_fit: str = Field(..., description="Reason for candidate fit to job.")

class JobListings(BaseModel):
    job_listings: List[JobListing] = Field(..., description="List of job listings")

class ScoredListing(BaseModel):
    job_listing: JobListing = Field(..., description="Job listing of interest")
    score: float = Field(..., description="Score of job listing fit to candidate profile on a scale of 1-10.")
    justfication: str = Field(..., description="Justification for score")

class ScoredListings(BaseModel):
    scored_listings: List[ScoredListing] = Field(..., description="List of scored job listings")

class CoverLetter(BaseModel):
    scored_listing: ScoredListing = Field(..., description="Scored job listing of interest")
    cover_letter: str = Field(..., description="Cover letter for scored listing")

class CoverLetters(BaseModel):
    cover_letters: List[CoverLetter] = Field(..., description="List of cover letters")
    candidate_profile: CandidateProfile

## Writing a custom tool to load in the resume!

In [9]:
from crewai.tools import BaseTool
from llama_parse import LlamaParse
from typing import Any
import nest_asyncio
nest_asyncio.apply()

class ResumeParser(BaseTool):
    """This tool parses a resume using Llama Parse and returns them into
    string form."""
    
    name: str = "Resume reader"
    description: str = "Parses a pdf resume file neatly for LLM processing"
    parser: LlamaParse = LlamaParse(
        api_key = os.environ["LLAMA_CLOUD_API_KEY"],
        # use_vendor_multimodal_model=True,
        # vendor_multimodal_name="anthropic-sonnet-3.5",
        result_type="markdown"
    )
    
    def find_resume(self) -> str:
        """Method to search 2 levels up for resume folder"""
        
        __curdir__ = os.getcwd()
        depth = 0
        resume_dir_path = ""
        
        while (depth<3):    
            if "Resume" not in os.listdir(__curdir__):
                __curdir__ = os.path.abspath(os.path.join(__curdir__, os.path.pardir))
                depth += 1
            else:
                resume_dir_path = os.path.abspath(os.path.join(__curdir__, "Resume"))
                break
        
        return resume_dir_path
    
    def _run(self) -> Dict[str, Any]:
        """Returns the entire resume in string format."""
        
        file_path = self.find_resume()
        resume = os.path.join(
            file_path,
            os.listdir(file_path)[0]
        )
        docs = self.parser.load_data(file_path=resume)
        return {
            "resume": "\n\n".join([doc.text for doc in docs])
        }

In [38]:
tool = ResumeParser()
result = tool._run()

Started parsing the file under job_id 3f52ede0-188d-4c83-893b-b90677687b89


In [ ]:
display(Markdown(result['resume']))

##### Creating LlamaIndex Tavily Tool

In [10]:
from crewai_tools import LlamaIndexTool
from llama_index.tools.tavily_research import TavilyToolSpec

tavily_spec = TavilyToolSpec(api_key=os.environ["TAVILY_API_KEY"])
tavily_tool_list = tavily_spec.to_tool_list()
tavily_tools = [LlamaIndexTool.from_tool(tool) for tool in tavily_tool_list]

#### Creating our Crew, Agents and Tasks

In [11]:
profiler_agent = Agent(
    config = job_search_agents_config['profiler'],
    tools = [ResumeParser()]
)
recruiter = Agent(
    config=job_search_agents_config['recruiter'],
    tools = [*tavily_tools] 
)
matcher = Agent(
    config = job_search_agents_config['matcher'],
)
reporter = Agent(
    config=job_search_agents_config['reporter'],
)

In [12]:
profiling_task = Task(
    config = job_search_tasks_config['candidate_profiling_task'],
    agent = profiler_agent,
    output_pydantic = CandidateProfile,
    expected_output = "A clear candidate profile"
)
research_task = Task(
    config = job_search_tasks_config['research_job_listings_task'],
    agent = recruiter,
    context = [profiling_task],
    output_pydantic = JobListings,
    expected_output = "A list of job listings that fit the candidate profile"
)
scoring_task = Task(
    config = job_search_tasks_config['match_and_score_listings_task'],
    agent = matcher,
    context = [research_task, profiling_task],
    output_pydantic=ScoredListings,
    expected_output = "A list of top 3 job listings ranked with justification"
)
cover_letter_task = Task(
    config = job_search_tasks_config['cover_letter_task'],
    agent = reporter,
    context = [scoring_task, profiling_task],
    output_pydantic=CoverLetters,
    expected_output="A list of cover letters for the top 3 job listings"
)

In [13]:
job_crew = Crew(
    agents = [
        profiler_agent,
        recruiter,
        matcher,
        reporter
    ],
    tasks = [
        profiling_task,
        research_task,
        scoring_task,
        cover_letter_task
    ],
    verbose = True,
)

2025-01-22 00:10:10,976 - 8342343488 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
results = job_crew.kickoff(
    inputs = {
        'job_requirements': 'Generative AI related data scientist jobs or management positions.'
    }
)

In [18]:
from IPython.display import display, Markdown

In [ ]:
display(Markdown(results.pydantic.cover_letters[0].cover_letter))

Lim Hsien Yong
[Your Address]
[City, State, Zip]
[Email]
[Phone Number]
[Date]

Hiring Manager
Generative AI Careers
[Company Address]
[City, State, Zip]

Dear Hiring Manager,

I am writing to express my interest in the Lead Data Scientist position at Generative AI Careers, as advertised. With a Master of IT in Business (Artificial Intelligence Track) and over five years of hands-on experience in data science, I am excited about the opportunity to leverage my expertise in Generative AI to drive innovative projects at your esteemed company.

In my most recent role as a Senior Data Scientist, I led the project for developing a Defect Detection system in Flowcells. This involved designing and prototyping advanced instance segmentation models, utilizing a Siamese Network architecture to deliver precise defect identification, resulting in significant cost savings and enhanced image matching efficiency. This project honed my technical skills in Python and reinforced my ability to implement impactful data-driven solutions.

Another project I successfully spearheaded was the development of a Generative BI Assistant Application. This application enabled users to extract insights through natural language queries, showcasing my strong expertise in Natural Language Processing and application development. This initiative not only improved operational efficiency but also provided valuable insights tailored to user needs, demonstrating my commitment to prioritizing business impact.

I believe my technical depth in data science and AI, coupled with my leadership abilities, aligns perfectly with the requirements of the Lead Data Scientist role. I am passionate about harnessing the power of Generative AI to unlock new potential and drive meaningful change within organizations.

Thank you for considering my application. I look forward to the possibility of discussing how my experience and vision can contribute to the exciting projects at Generative AI Careers.

Sincerely,
Lim Hsien Yong

In [28]:
import pandas as pd

costs = 0.150 * (job_crew.usage_metrics.prompt_tokens + job_crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([job_crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0029


/var/folders/zb/r15p7t_d62d8m2s0623s22gh0000gn/T/ipykernel_24720/1113274036.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df_usage_metrics = pd.DataFrame([job_crew.usage_metrics.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,19507,14607,4096,4900,9


This is not the end though! Clearly we can do more with the cover letter!

## Training the crew!

In [ ]:
job_crew.train(
    inputs = {
         'job_requirements': 'Generative AI related data scientist jobs or management positions.'
    },
    n_iterations = 1,
    filename="titus_training.pkl"
)

In [ ]:
job_crew.test(
    inputs = {
         'job_requirements': 'Generative AI related data scientist jobs or management positions.'
    },
    n_iterations = 1,
)

Looks good!

In [ ]:
results = job_crew.kickoff(
    inputs = {
        'job_requirements': 'Generative AI related data scientist jobs or management positions.'
    }
)

In [14]:
from IPython.display import display, Markdown

In [25]:
display(Markdown(results.pydantic.cover_letters[0].cover_letter))

Dear Hiring Manager,

I am writing to express my enthusiasm for the Data Scientist position within the Generative AI Innovation Center at Amazon. With a Master's degree in IT in Business specialized in Artificial Intelligence from Singapore Management University and five years of hands-on experience in data science, I am eager to contribute to your dynamic team in harnessing the potential of Generative AI.

During my tenure as a Senior Data Scientist, I successfully prototyped and deployed sophisticated machine learning models in a project titled 'Flowcell Defect Detection'. This initiative not only facilitated accurate defect identification but also yielded significant cost savings for the organization. The depth of my technical acumen in machine learning and data engineering, combined with my proficiency in microservices architecture, ensures I am well-equipped to tackle the challenges presented by this role.

Additionally, I spearheaded the development of a 'Generative BI Assistant Application', leveraging natural language processing. This project enhanced user interaction with complex datasets, optimizing communication and decision-making. This experience aligns seamlessly with the collaborative and problem-solving ethos that your team embodies.

I am highly motivated by the opportunity to help AWS customers leverage innovative generative AI solutions, driving transformational business outcomes. I am confident that my technical skills, combined with a problem-solving mindset and a passion for collaboration, will allow me to make a meaningful contribution to Amazon's mission.

Thank you for considering my application. I look forward to the opportunity to discuss how I can further enhance your team.

Sincerely,
Lim Hsien Yong

# 2. The Job Interview Preparation Crew
Now let's use the information we have from the previous crew to help us prepare for the job interview in advance!

In [15]:
job_interview_agents_config, job_interview_tasks_config = load_configs(
    {
        'agents': '../config/job_interview_agents.yml',
        'tasks': '../config/job_interview_tasks.yml',
    }
)

In [16]:
class JobProfile(BaseModel):
    job_title: str = Field(..., description="Job title")
    company_name: str = Field(..., description="Company name")
    job_description: str = Field(..., description="Job description")
    required_technical_skills: List[str] = Field(..., description="List of required technical skills. For example 'machine learning'.")
    required_non_technical_skills: List[str] = Field(..., description="List of required non-technical skills. For example 'strategic planning'.")
    years_of_experience_required: int = Field(..., description="Years of experience required")

class InterviewQuestion(BaseModel):
    company: str = Field(..., description="Hiring company name")
    job_title: str = Field(..., description="Job title")
    question: str = Field(..., description="Interview question to ask candidate")
    skill_required: str = Field(..., description="What the interviewer is looking out for")
    sample_answer: str = Field(..., description="Sample answer to question based on candidate's profile")

class InterviewPrep(BaseModel):
    interview_questions: List[InterviewQuestion] = Field(..., description="List of interview questions")

class ElevatorPitch(BaseModel):
    interview_prep: InterviewPrep = Field(..., description="Interview Q&A doocument")
    elevator_pitch: str = Field(..., description="Customized elevator pitch for the job description to answer the question 'tell me more about yourself'.")

In [17]:
interviewer = Agent(
    config=job_interview_agents_config['interviewer'],
)
pitcher = Agent(
    config=job_interview_agents_config['pitcher'],
)

interview_prep_task = Task(
    config = job_interview_tasks_config['interview_prep_task'],
    agent = interviewer,
    output_pydantic = InterviewPrep,
    expected_output = "A compiled list of interview questions and answers for each job listing"
)
elevator_pitch_task = Task(
    config = job_interview_tasks_config['conversation_starter_task'],
    agent = pitcher,
    output_pydantic = ElevatorPitch,
    context = [interview_prep_task],
    expected_output = "A customized elevator pitch for the job listing."
)

In [18]:
interview_crew = Crew(
    agents = [
        interviewer,
        pitcher
    ],
    tasks = [
        interview_prep_task,
        elevator_pitch_task,
    ],
    verbose = True,
)

2025-01-22 00:10:33,042 - 8342343488 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [81]:
candidate_profile = results.pydantic.candidate_profile.model_dump()

In [90]:
scored_listings =  [
    result.scored_listing.model_dump()['job_listing'] for result in results.pydantic.cover_letters
]

In [ ]:
interview_crew_inputs = [
    {
        "candidate_profile": candidate_profile,
        "job_listing": listing
    } for listing in scored_listings
]

In [95]:
interview_results = interview_crew.kickoff_for_each(
    inputs = interview_crew_inputs,
)

2025-01-21 23:13:49,674 - 8342343488 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Interview preparation agent
## Task: Prepare a comprehensive interview preparation document for the candidate's preparation process. Develop at least 10 questions and sample answers based on the job requirements and the candidate's profile.
candidate_profile: {'name': 'Lim Hsien Yong', 'highest_qualification': 'Masters Degree', 'field_of_study': 'Artificial Intelligence', 'institution_of_study': 'Singapore Management University', 'technical_skills': ['Data Science', 'Machine Learning', 'Natural Language Processing', 'Computer Vision', 'Generative AI', 'Graph Analytics', 'Cloud Computing (AWS, GCP, Azure)', 'Database Management (SQL, MongoDB, Neo4j)', 'API Development', 'Web Applications Development'], 'non_technical_skills': ['Problem Solving', 'Project Management', 'Collaboration', 'Communication', 'Research and Development', 'Innovation', 'Customer Insight', 'Data Analysis'], 'years_of_experience': 5, 'current_job_title': 'Senior Data Scientist', 'work_experiences': [{'proje

2025-01-21 23:14:20,755 - 8342343488 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Interview preparation agent
## Final Answer: 
{
  "interview_questions": [
    {
      "company": "Amazon",
      "job_title": "Data Scientist, Generative AI Innovation Center",
      "question": "Can you explain a complex project where you utilized Generative AI, and what was your role?",
      "skill_required": "Generative AI",
      "sample_answer": "In my project 'AI-powered Troubleshooting Assistant', I developed an AI assistant that leveraged generative AI for real-time equipment troubleshooting. My role involved designing the architecture as well as developing the algorithms that enabled the AI to understand user queries and provide accurate solutions. The project not only improved the response time for our line engineers but also enhanced the overall operational efficiency, which was pivotal for the team."
    },
    {
      "company": "Amazon",
      "job_title": "Data Scientist, Generative AI Innovation Center",
      "question": "What is your experience with cloud

2025-01-21 23:14:49,212 - 8342343488 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types
2025-01-21 23:14:49,226 - 8342343488 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


...



# Agent: Master elevator pitch specialist
## Final Answer: 
{
  "interview_prep": {
    "interview_questions": [
      {
        "company": "EY",
        "job_title": "Data Scientist (Gen AI), Manager, Data & AI, Technology Consulting",
        "question": "Can you elaborate on a project where you implemented Generative AI solutions?",
        "skill_required": "Generative AI",
        "sample_answer": "In my previous project, I developed an AI-powered troubleshooting assistant that leveraged generative AI for real-time equipment diagnostics. This involved training the model on historical data, enabling it to provide immediate troubleshooting support for line engineers. The outcome was a significant reduction in equipment downtime and operational costs."
      },
      {
        "company": "EY",
        "job_title": "Data Scientist (Gen AI), Manager, Data & AI, Technology Consulting",
        "question": "Describe your experience with Large Language Models.",
        "skill_required"

2025-01-21 23:16:16,503 - 8342343488 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Interview preparation agent
## Final Answer: 
{
  "interview_questions": [
    {
      "company": "Capital One",
      "job_title": "Manager, Data Scientist, Generative AI Systems",
      "question": "Can you describe a project where you utilized Generative AI techniques?",
      "skill_required": "Generative AI",
      "sample_answer": "In my previous role, I developed an AI-powered troubleshooting assistant that utilized generative AI for real-time equipment troubleshooting. This project involved creating a model that could generate actionable insights based on historical data. It optimized operational efficiency for line engineers by effectively predicting and diagnosing equipment issues. The successful deployment of this project resulted in a notable reduction in downtime, significantly benefiting overall productivity."
    },
    {
      "company": "Capital One",
      "job_title": "Manager, Data Scientist, Generative AI Systems",
      "question": "What strategic appro

2025-01-21 23:16:39,579 - 8342343488 - __init__.py-__init__:100 - WARNING: Invalid type TaskOutput for attribute 'output.value' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types




# Agent: Master elevator pitch specialist
## Final Answer: 
{
  "interview_prep": {
    "interview_questions": [
      {
        "company": "Capital One",
        "job_title": "Manager, Data Scientist, Generative AI Systems",
        "question": "Can you describe a project where you utilized Generative AI techniques?",
        "skill_required": "Generative AI",
        "sample_answer": "In my previous role, I developed an AI-powered troubleshooting assistant that utilized generative AI for real-time equipment troubleshooting. This project involved creating a model that could generate actionable insights based on historical data. It optimized operational efficiency for line engineers by effectively predicting and diagnosing equipment issues. The successful deployment of this project resulted in a notable reduction in downtime, significantly benefiting overall productivity."
      },
      {
        "company": "Capital One",
        "job_title": "Manager, Data Scientist, Generative AI 

In [101]:
display(Markdown(interview_results[0].pydantic.elevator_pitch))

Hello! I'm a passionate data scientist with specialized experience in generative AI. At my last position, I led a project that developed an AI-powered troubleshooting assistant, improving our operational response time significantly. I have a strong background in using cloud platforms like AWS, utilizing them to streamline data processing in various projects, including forecasting electricity demand which enhanced profitability by 20%. My collaborative approach fuels innovation as I work closely with teams and incorporate user feedback to refine solutions. I am committed to continuous learning, actively engaging in AI forums and courses to stay at the cutting edge of technology. I’m excited about the opportunity at Amazon’s Generative AI Innovation Center, where I can bring my expertise and contribute to groundbreaking projects that redefine user experiences. Thank you!

In [96]:
import pandas as pd

costs = 0.150 * (interview_crew.usage_metrics.prompt_tokens + interview_crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([interview_crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0023


/var/folders/zb/r15p7t_d62d8m2s0623s22gh0000gn/T/ipykernel_42474/63002604.py:7: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df_usage_metrics = pd.DataFrame([interview_crew.usage_metrics.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,15493,7355,0,8138,6


# 3. Combing both crews into a single flow
Allows us to execute everything in one swoop

In [20]:
from crewai import Flow
from crewai.flow.flow import listen, start

In [31]:
from typing import Optional
import json

class JobSearchPipeline(Flow):
    @start()
    def job_crew_kickoff(self, inputs: Optional[Dict[str, Any]]= None):
        print("Job crew tasks kickoff...")
        job_crew_results = job_crew.kickoff(inputs=inputs)
        results= job_crew_results.pydantic.model_dump()
        print(results)
        return results
    
    @listen(job_crew_kickoff)
    def log_job_results(self, results: Dict[str, Any]):
        with open("job_crew_results.json", "w") as file:
            json.dump(results, file, indent=4)
        print("Results saved to './job_crew_results.json'")
    
    @listen(job_crew_kickoff)
    def postprocess_job_crew_results(self, results: Dict[str, Any]):
        print("Preparing interview crew inputs...")
        candidate_profile = results['candidate_profile']
        scored_listings =  [
            result['scored_listing']['job_listing'] for result in results['cover_letters']
        ]
        interview_crew_inputs = [
            {
                "candidate_profile": candidate_profile,
                "job_listing": listing
            } for listing in scored_listings
        ]
        return interview_crew_inputs
    
    @listen(postprocess_job_crew_results)
    def log_interview_crew_inputs(self, interview_crew_inputs: List[Dict[str, Any]]):
        with open("interview_crew_inputs.json", "w") as file:
            json.dump(interview_crew_inputs, file, indent=4)
        print("interview crew input saved!")
    
    @listen(postprocess_job_crew_results)
    def interview_crew_kickoff(self, interview_crew_inputs: List[Dict[str, Any]]):
        print("Interview crew tasks kickoff...")
        interview_crew_results = interview_crew.kickoff_for_each(inputs=interview_crew_inputs)
        return interview_crew_results

In [32]:
flow = JobSearchPipeline()
flow.plot()

Plot saved as crewai_flow.html


In [23]:
from IPython.display import IFrame

IFrame(src="crewai_flow.html", width="150%", height=600)

In [ ]:
results = flow.kickoff(
    inputs = {
        'job_requirements': 'Generative AI related data scientist jobs or management positions.'
    }
)

In [36]:
for result in results:
    display(Markdown(result.pydantic.elevator_pitch))

Hello, I'm [Your Name], a passionate Senior Data Scientist with extensive experience in leveraging machine learning and AI to drive impactful data-driven solutions. In my previous roles, I've successfully developed predictive models for defect detection, improved data analysis workflows, and led cross-functional teams to deliver cutting-edge AI applications. I’m particularly skilled in strategic planning and team collaboration, striving to align project goals with innovative approaches. With a strong focus on continuous learning and adapting to new technologies, I’m excited about the opportunity to contribute to Google’s mission of harnessing AI to create transformative solutions that enhance user experiences.

Hello! I’m a passionate and experienced Senior Solutions Architect specializing in AI and ML with a robust background in cloud technologies. I have successfully led various machine learning projects, utilizing AWS to develop scalable solutions that drove significant cost savings and enhanced decision-making capabilities for clients. My collaborative mindset has allowed me to effectively engage with stakeholders, ensuring that the solutions I develop not only meet technical requirements but also align with user needs. I stay ahead of industry trends through continuous learning and involvement in key conferences, empowering me to tackle complex challenges, such as data privacy, and create innovative, high-quality AI solutions that yield measurable outcomes.

Hi, I’m [Your Name], a Senior Data Scientist with over 5 years of experience in building impactful machine learning models and data-driven solutions. My journey has enabled me to work with renowned companies, where I successfully led projects like electricity spot price forecasting with a model accuracy of over 94%, resulting in optimized trading strategies that saved costs. My passion lies in leveraging cloud technologies such as Azure for deploying scalable applications and collaborating with cross-functional teams to drive innovations, exemplified in real-time IoT monitoring systems. I'm committed to staying at the forefront of data science trends to deliver cutting-edge solutions that can transform business processes. I would love to bring my expertise in machine learning and data engineering to Microsoft, contributing to your innovative projects.